# 如果有 GPU，确保 pytorch 是 cuda 版本的

pip3 install torch 

它是一个高性能的数学计算库,“张量（Tensor）”就是它的核心概念

# 安装 mppi 库

pip3 install pytorch-mppi 

这是一个别人（UM-ARM-Lab）写好的、基于 PyTorch 的 MPPI 算法实现库

如何生成符合高斯分布的随机噪声？
如何把噪声施加到动力学模型上进行积分（Rollouts）？
如何根据 Cost 计算每一条轨迹的权重（指数加权）？
如何更新控制量并平滑输出？

In [7]:
import time
import torch

print("====== 基本信息 ======")
print("PyTorch 版本:", torch.__version__)
print("编译时 CUDA 版本:", torch.version.cuda)
print("CUDA 是否可用:", torch.cuda.is_available())
print("GPU 数量:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("默认 GPU:", torch.cuda.get_device_name(0))

print("\n====== cuDNN 信息 ======")
print("cuDNN 是否可用:", torch.backends.cudnn.enabled)
print("cuDNN 版本:", torch.backends.cudnn.version())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("\n使用设备:", device)

print("\n====== 矩阵乘法性能测试 ======")
a = torch.randn((2000, 2000), device=device)
b = torch.randn((2000, 2000), device=device)

if device.type == "cuda":
    torch.cuda.synchronize()
t0 = time.time()
c = a @ b
if device.type == "cuda":
    torch.cuda.synchronize()
t1 = time.time()

print("结果张量设备:", c.device)
print("矩阵乘法耗时: {:.4f} 秒".format(t1 - t0))


====== 基本信息 ======
PyTorch 版本: 2.4.1+cu121
编译时 CUDA 版本: 12.1
CUDA 是否可用: True
GPU 数量: 1
默认 GPU: NVIDIA GeForce RTX 3060 Laptop GPU

====== cuDNN 信息 ======
cuDNN 是否可用: True
cuDNN 版本: 90100

使用设备: cuda

====== 矩阵乘法性能测试 ======
结果张量设备: cuda:0
矩阵乘法耗时: 0.0033 秒


根据代码结构，三层架构的输入输出如下：

## 三层架构输入输出



┌─────────────────────────────────────────────────────────────────────────────┐
│                        第1层: GlobalPathPlanner (全局路径规划)                │
├─────────────────────────────────────────────────────────────────────────────┤
│  输入:                                                                       │
│    • start: (x, y) 起点世界坐标                                              │
│    • goal:  (x, y) 目标点世界坐标                                            │
│    • map:   OccupancyGrid 栅格地图 (来自 /map 话题)                          │
├─────────────────────────────────────────────────────────────────────────────┤
│  输出:                                                                       │
│    • path:  [(x₀,y₀), (x₁,y₁), ..., (xₙ,yₙ)] 全局路径点序列 (世界坐标)       │
└─────────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────────┐
│                        第2层: TrajectoryGenerator (轨迹生成)                  │
├─────────────────────────────────────────────────────────────────────────────┤
│  输入:                                                                       │
│    • path:       [(x,y), ...] 全局路径点 (来自第1层)                         │
│    • current_pose: (x, y, θ) 当前位姿                                        │
│    • v_max:      最大线速度 (默认 0.22 m/s)                                  │
│    • a_max:      最大加速度 (默认 0.5 m/s²)                                  │
├─────────────────────────────────────────────────────────────────────────────┤
│  输出:                                                                       │
│    • trajectory: [(t, x, y, θ, v, ω), ...] 时间参数化轨迹                    │
│                   t=时间, (x,y,θ)=位姿, v=线速度, ω=角速度                   │
└─────────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────────┐
│                        第3层: MPCTracker (MPC跟踪控制)                        │
├─────────────────────────────────────────────────────────────────────────────┤
│  输入:                                                                       │
│    • current_state: (x, y, θ) 当前位姿 (来自 /odom)                          │
│    • ref_window:    [(x,y,θ,v,ω), ...] 参考轨迹窗口 (N+1个点, 来自第2层)     │
├─────────────────────────────────────────────────────────────────────────────┤
│  输出:                                                                       │
│    • v:     线速度控制量 (m/s)                                               │
│    • ω:     角速度控制量 (rad/s)                                             │
│    → 发布到 /cmd_vel 话题控制机器人                                          │
└─────────────────────────────────────────────────────────────────────────────┘



## 数据流示例



In [ ]:
# 第1层: 几何路径 (无时间信息)
path = [(0,0), (0.5,0), (1.0,0.3), (1.5,0.8), (2.0,1.0)]  # 5个航点

# 第2层: 时间参数化轨迹 (插值+速度规划)
trajectory = [
    (0.0,  0.00, 0.00, 0.00, 0.10, 0.00),  # t=0s
    (0.1,  0.01, 0.00, 0.00, 0.12, 0.00),  # t=0.1s
    (0.2,  0.02, 0.00, 0.01, 0.15, 0.02),  # t=0.2s
    ...  # 约100个点
]

# 第3层: 控制指令
v, omega = 0.18, 0.05  # 发布到 /cmd_vel



## 关键接口

| 层级 | 方法 | 输入 | 输出 |
|------|------|------|------|
| 1 | `planner.plan(start, goal)` | 起点、终点坐标 | 路径点列表 |
| 2 | `generator.generate(path)` | 路径点列表 | 时间参数化轨迹 |
| 2 | `generator.get_reference_window(t, N)` | 当前时间、窗口大小 | N+1个参考点 |
| 3 | `tracker.solve(state, ref_window)` | 当前状态、参考窗口 | (v, ω, 预测轨迹) |

In [ ]:
# 发布自定义目标点到 ROS 话题 /custom_goal
rostopic pub /custom_goal geometry_msgs/PoseStamped "header: {frame_id: 'map'}
pose: {position: {x: 0.44, y: 0.47, z: 0}, orientation: {w: 1}}"

rosrun simple_mppi plot_metrics.py _algo:=MPPI # 可视化MPPI算法指标

数据正常，且**符合各算法特性**。分析如下：

## 数据汇总

| 算法 | 路径长度 | 平均速度 | 平均CTE | 最大CTE |
|------|---------|---------|---------|---------|
| **MPC** | 2.636m ✓最短 | 0.111 m/s ✓最快 | 0.017m ✓最精 | 0.418m |
| MPPI | 2.811m | 0.088 m/s | 0.070m | 0.342m ✓最小 |
| iLQR | 2.832m | 0.087 m/s | 0.073m | 0.474m |

## 是否符合算法特性？

### ✅ MPC（模型预测控制）
- **平均CTE最小 (0.017m)** — 符合预期，MPC 使用精确的优化求解器(IPOPT)，跟踪精度最高
- **速度最快、路径最短** — MPC 的代价函数直接优化轨迹误差，能更高效地沿参考路径行驶
- **最大CTE中等** — 优化器需要几步收敛，起步时可能有短暂偏差

### ✅ MPPI（采样预测控制）
- **最大CTE最小 (0.342m)** — 符合预期，MPPI 采样探索性强，不容易陷入局部极端偏差
- **平均CTE中等** — 采样方法本质是概率加权，精度不如确定性优化
- **路径稍长** — 采样噪声导致轨迹不如 MPC 直接

### ✅ iLQR（迭代线性二次调节器）
- **性能中庸** — iLQR 是 MPC 的简化版（线性化+LQR），精度介于 MPC 和 MPPI 之间
- **最大CTE最大** — iLQR 对初始猜测敏感，起步阶段可能收敛慢
- **计算效率高**（虽然这里没测时间，但 iLQR 理论上最快）

## 结论

| 优势 | 算法 |
|------|------|
| **跟踪精度** | MPC > iLQR ≈ MPPI |
| **鲁棒性/探索性** | MPPI > MPC > iLQR |
| **计算效率** | iLQR > MPC > MPPI |

数据**完全符合理论预期**：
- 想要最精确的轨迹跟踪 → 用 **MPC**
- 想要更好的鲁棒性（动态环境） → 用 **MPPI**
- 想要快速计算（资源受限） → 用 **iLQR**